In [ ]:
using HDF5
using PyPlot
using LsqFit

In [ ]:
h5file = "2d_out.h5"
names = ["Gt","mq_sqrt3_corr", "mq_q0_corr", "afvc_corr", "fvc_corr","m_120degs_corr"]

In [ ]:
function out_to_txt(h5file,names)

    fid = h5open(h5file,"r")
    temps = fid["temperatures"]
    num_temps = length(temps)
    for iname in names
        data = fid["$(iname)/mean"]
        name = split(iname,"_corr")[1]
        mc_steps = length(data[:,1])
        for it in 1:num_temps
            open("$(name)_$(it).dat","w") do fp
                for i in 1:mc_steps
                    println(fp,i," ",data[:,it][i])
                end
            end
        end
    end
    close(fid)
end

out_to_txt(h5file,names)


function compute_τ(tdatas,ydatas,p0)
    model(t,p) = p[1] .- (1/p[2])*t
    fit = curve_fit(model, tdatas, ydatas, p0)
    fit.param[2]
end



function compute_Tc(temps,τs,p0)
    model(t,p) = p[1] .+ p[2] ./ sqrt.(t .- p[3])
    fit = curve_fit(model,temps,log.(τs),p0)
    fit.param[3]
end


function execute(fid,name)

    temps = fid["temperatures"]
    num_temps = length(temps)
    @assert num_temps > 1
    ydata = fid["$(name)/mean"]
    tdata = [i for i in 1:length(ydata[:,1])]
    p01 = [1.0,1.0]
    τs = Vector{Float64}(undef,num_temps)
    println("τs=$(τs)")
    
    for it in 1:num_temps
        temp_ydata = log.(abs.(ydata[:,it]))
        τs[it] = compute_τ(tdata,temp_ydata,p01)
    end
    
    p02 = [1.0,1.0,1.0]
    Tc = compute_Tc(collect(temps),τs,p02)
    Tc
    
end

#=
h5file = "2d_out.h5"
fid = h5open(h5file,"r")
name = "Gt"
Tc_of_Gt = execute(fid,name)
println("Tc of Gt = $(Tc_of_Gt)")